In [1]:
from json import loads, dumps
from pathlib import Path
from random import choice

In [2]:
from os import environ
from pathlib import Path
from json import loads, dumps
from random import choice
environ["OPENAI_API_KEY"] = Path("~/.openaiapikey").expanduser().read_text().strip()
from openai import OpenAI
from random import randint

openaiClient = OpenAI()
def gpt_3_5_turbo_completion(query, temperature = 1):
    answer = openaiClient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        temperature = temperature,
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def gpt_4_turbo_completion(query, temperature = 1):
    answer = openaiClient.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        temperature = temperature,
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def tryRecieveAnswer(query, completionFunction = gpt_3_5_turbo_completion, answerConversion = lambda x: True, maxTries = 10, temperature = 1):
    tryNumber = 0
    while tryNumber < maxTries:
        answer = completionFunction(query, temperature)
        try:
            answer = answerConversion(answer)
            return (answer, True)
        except:
            pass
        tryNumber += 1
    print(f"Failed to recieve answer for query: {query}")
    return (None, False)

def listAnswerConversion(answer):
    result = loads(answer)
    assert isinstance(result, list)
    for item in result:
        assert isinstance(item, str)
    return result

In [12]:
mastgPath = Path("../master-database-files/master-adapted-subtopic-tree-generation")
termlistPath = mastgPath / "technical_terms.txt"
mePath = Path("../master-database-files/master-experimental/subtopic_tree_comparison")
progressPath = mePath / "progress.txt"
meTreesPath = mePath / "trees"
frAss5Path = meTreesPath / "frAss5"

In [4]:
def hasTermAsLeaf(term, tree):
    if tree == None:
        return False
    for name, childs in tree.items():
        if name == term and childs == None:
            return True
        if hasTermAsLeaf(term, childs):
            return True
    return False

In [5]:
def findMatchingTopic(term, choices):
    if len(choices) == 1:
        return choices[0]
    numberedChoicesString = "{" + ", ".join([f'{i}: "{x}"' for i, x in enumerate(choices)]) + "}"
    query = f'Which of the topics {numberedChoicesString} is most likely to contain "{term}" as a subtopic? Return only the number without description.' 
    def answerConversion(answer):
        r = int(answer)
        return choices[r]  
    answer, success = tryRecieveAnswer(query, answerConversion = answerConversion)  
    if not success:
        answer = choice(choices)
    return answer

In [8]:
def generateSubtopicTreeWithFreeAssociationAlgorithm(rootTopic, targetPath, termListPath, maxBranchNumber, targetBranchNumber):
    termlist = termListPath.read_text().split("\n")
    if not targetPath.exists():
        targetPath.mkdir()
    if not (targetPath / "subtopic_tree.json").exists():
        targetPath.joinpath("subtopic_tree.json").write_text(dumps({rootTopic: {}}))
    subtopicTree = loads(targetPath.joinpath("subtopic_tree.json").read_text())
    for termIndex, term in enumerate(termlist):
        progressPath.write_text(f"{termIndex}/{len(termlist)}")
        if hasTermAsLeaf(term, subtopicTree):
            continue
        # Add term to the tree
        # Search place to insert term
        currentBranch = subtopicTree
        currentTopic = rootTopic
        while any([type(child) == dict for child in currentBranch.values()]):
            choices = [name for name, childs in currentBranch.items() if type(childs) == dict]
            answer = findMatchingTopic(term, choices)
            currentBranch = currentBranch[answer]
            currentTopic = answer
        # Calculate children if necessary
        if len(currentBranch) >= maxBranchNumber:
            children = [*currentBranch.keys()]
            for child in children:
                currentBranch.pop(child)
            children.append(term)
            query = f'In which {targetBranchNumber} subtopics can the topic "{currentTopic}" be divided? Return nothing but the list of subtopics formatted as: ["subtopic1", "subtopic2", ...]'
            answer, success = tryRecieveAnswer(query, answerConversion = listAnswerConversion)
            if success:
                for c in answer:
                    currentBranch[c] = {}
                for c in children:
                    currentBranch[findMatchingTopic(c, answer)][c] = None
            else:
                for c in children:
                    currentBranch[c] = None
        else:
            currentBranch[term] = None
        targetPath.joinpath("subtopic_tree.json").write_text(dumps(subtopicTree, indent = 1))


In [15]:
termlist = termlistPath.read_text().split("\n")
subtopicTree = loads(frAss5Path.joinpath("subtopic_tree.json").read_text())
def removeTermFromTree(tree, termlist):
    leafs = [term for term, value in tree.items() if value == None]
    for term in leafs:
        if not term in termlist:
            tree.pop(term)
    for term, value in tree.items():
        if value != None:
            removeTermFromTree(value, termlist)
removeTermFromTree(subtopicTree, termlist)
frAss5Path.joinpath("subtopic_tree2.json").write_text(dumps(subtopicTree, indent = 1))

52144

In [13]:
generateSubtopicTreeWithFreeAssociationAlgorithm("Physics", frAss5Path, termlistPath, 5, 5)

KeyboardInterrupt: 

In [ ]:
def findMatchingTopicIfAny(term, choices):
    if len(choices) == 1:
        query = f'Is the topic {choices[0]} including "{term}" as a subtopic? Return 1 if yes, 0 if no.'
    else:
        numberedChoicesString = "{" + ", ".join([f'{i + 1}: "{x}"' for i, x in enumerate(choices)]) + "}"
        query = f'Which of the topics {numberedChoicesString} is most likely to contain "{term}" as a subtopic? Return only the number without description. Return 0 if none.' 
    def answerConversion(answer):
        r = int(answer)
        if r == 0:
            return None
        return choices[r - 1]
    answer, success = tryRecieveAnswer(query, answerConversion = answerConversion)
    if not success:
        return None
    return answer

In [ ]:
def generateSubtopicTreeWithSubdivisionAlgorithm(rootTopic, targetPath, termListPath, maxBranchNumber):
    termlist = termListPath.read_text().split("\n")
    if not targetPath.exists():
        targetPath.mkdir()
    if not (targetPath / "subtopic_tree.json").exists():
        targetPath.joinpath("subtopic_tree.json").write_text(dumps({rootTopic: {}}))
    subtopicTree = loads(targetPath.joinpath("subtopic_tree.json").read_text())
    for term in termlist:
        if hasTermAsLeaf(term, subtopicTree):
            continue
        # Add term to the tree
        # Search place to insert term
        currentBranch = subtopicTree
        currentTopic = rootTopic
        while any([type(child) == dict for child in currentBranch.values()]):
            choices = [name for name, childs in currentBranch.items() if type(childs) == dict]
            answer = findMatchingTopic(term, choices)
            currentBranch = currentBranch[answer]
            currentTopic = answer
        # Calculate children if necessary
        if len(currentBranch) >= maxBranchNumber:
            children = [*currentBranch.keys()]
            for child in children:
                currentBranch.pop(child)
            children.append(term)
            for c in children:
                currentBranch[c] = None
        else:
            currentBranch[term] = None
        targetPath.joinpath("subtopic_tree.json").write_text(dumps(subtopicTree, indent = 1))